In [5]:
import requests
import urllib3
import pandas as pd
import panel as pn
import folium
from datetime import datetime, timedelta
pn.extension("tabulator")

import hvplot.pandas

%run sensitive.ipynb

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [6]:
auth_url = "https://www.strava.com/oauth/token"
activities_url = "https://www.strava.com/api/v3/athlete/activities?after="



print("requesting Token... \n")
res = requests.post(auth_url, data=payload_jando, verify=False)
access_token = res.json()["access_token"]

header = {"Authorization": "Bearer " + access_token}
param = {"per_page": 200, "page": 1}
activities = requests.get(activities_url, headers=header, params=param).json()
print("Got Token")

requesting Token... 

Got Token


In [7]:
df = pd.json_normalize(activities)
df.columns

Index(['resource_state', 'name', 'distance', 'moving_time', 'elapsed_time',
       'total_elevation_gain', 'type', 'sport_type', 'workout_type', 'id',
       'start_date', 'start_date_local', 'timezone', 'utc_offset',
       'location_city', 'location_state', 'location_country',
       'achievement_count', 'kudos_count', 'comment_count', 'athlete_count',
       'photo_count', 'trainer', 'commute', 'manual', 'private', 'visibility',
       'flagged', 'gear_id', 'start_latlng', 'end_latlng', 'average_speed',
       'max_speed', 'has_heartrate', 'heartrate_opt_out',
       'display_hide_heartrate_option', 'elev_high', 'elev_low', 'upload_id',
       'upload_id_str', 'external_id', 'from_accepted_tag', 'pr_count',
       'total_photo_count', 'has_kudoed', 'athlete.id',
       'athlete.resource_state', 'map.id', 'map.summary_polyline',
       'map.resource_state', 'average_heartrate', 'max_heartrate',
       'average_watts', 'kilojoules', 'device_watts'],
      dtype='object')

In [12]:
activity_ids = df["id"]
detailed_activities = pd.DataFrame()
for id in activity_ids: 
    detailed_url = f"https://www.strava.com/api/v3/activities/{id}"
    new_data = requests.get(detailed_url, headers=header, params={"include_all_efforts": "true"}).json()
    new_data = pd.json_normalize(new_data)
    detailed_activities = pd.concat([detailed_activities, new_data], ignore_index = True)

C:\Users\jackl\AppData\Local\Temp\ipykernel_13456\3179496513.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  detailed_activities = pd.concat([detailed_activities, new_data], ignore_index = True)
C:\Users\jackl\AppData\Local\Temp\ipykernel_13456\3179496513.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  detailed_activities = pd.concat([detailed_activities, new_data], ignore_index = True)
C:\Users\jackl\AppData\Local\Temp\ipykernel_13456\3179496513.py:7: FutureWarning: The behavior of Data

In [13]:
detailed_activities.columns

Index(['resource_state', 'name', 'distance', 'moving_time', 'elapsed_time',
       'total_elevation_gain', 'type', 'sport_type', 'workout_type', 'id',
       'start_date', 'start_date_local', 'timezone', 'utc_offset',
       'location_city', 'location_state', 'location_country',
       'achievement_count', 'kudos_count', 'comment_count', 'athlete_count',
       'photo_count', 'trainer', 'commute', 'manual', 'private', 'visibility',
       'flagged', 'gear_id', 'start_latlng', 'end_latlng', 'average_speed',
       'max_speed', 'has_heartrate', 'heartrate_opt_out',
       'display_hide_heartrate_option', 'elev_high', 'elev_low', 'upload_id',
       'upload_id_str', 'external_id', 'from_accepted_tag', 'pr_count',
       'total_photo_count', 'has_kudoed', 'description', 'calories',
       'perceived_exertion', 'prefer_perceived_exertion', 'segment_efforts',
       'splits_metric', 'splits_standard', 'laps', 'best_efforts',
       'stats_visibility', 'hide_from_home', 'device_name', 'embed_

In [14]:
all_detailed = detailed_activities[['name', 'distance', 'moving_time', 'elapsed_time',
       'total_elevation_gain', 'type', 'sport_type', 'workout_type', 'id','start_date_local', 'start_latlng', 'end_latlng', 'average_speed',
       'max_speed', 'has_heartrate', 'elev_high', 'elev_low', 'pr_count','calories',
       'perceived_exertion', 'prefer_perceived_exertion', 'segment_efforts',
       'splits_metric', 'splits_standard', 'laps', 'best_efforts', 'map.id',
       'map.polyline', 'map.resource_state', 'map.summary_polyline', 'average_heartrate',
       'max_heartrate', 'gear.name', 'gear.distance']][detailed_activities["type"] == "Run"]
all_detailed = all_detailed.rename(columns={'start_date_local': 'date'})
all_detailed['date'] = all_detailed['date'].apply(lambda x: x.replace('T', ' ', 1).replace('Z', ' ', 1))
all_detailed['date'] = pd.to_datetime(all_detailed['date']) #.apply(lambda x: dt.strptime(x, '%Y-%m-%d'))
date_range = pd.date_range(start=datetime.today()-timedelta(days=365),end=datetime.today())
one_week_ago = date_range - timedelta(days=7)
four_weeks_ago = date_range - timedelta(days=28)

def ac_ratio(workload, acute, chronic):
    acwr = pd.Series()
    for index in range(len(date_range)):
        acwr[index] = ((sum(workload.iloc[:,0][workload['date'] > acute[index]])
         / ((sum(workload.iloc[:,0][workload['date'] > chronic[index]])) / 4)))
    return acwr    

dist_workload = all_detailed[['distance', 'date']]
all_detailed['RPE'] = all_detailed['elapsed_time'] * all_detailed['perceived_exertion']
RPE_workload = all_detailed[['RPE', 'date']]
RPE_workload['RPE'] = RPE_workload['RPE'].fillna(0)

dist_AC_ratio = ac_ratio(dist_workload, one_week_ago, four_weeks_ago)
RPE_AC_ratio = ac_ratio(RPE_workload, one_week_ago, four_weeks_ago)

all_detailed['difficulty_index'] = (all_detailed['distance']) * (all_detailed['total_elevation_gain'] + 1) #### need to work on this...
#all_detailed[['difficulty_index', 'perceived_exertion']]

workload_df = pd.DataFrame({"Distance ACWR": dist_AC_ratio, "RPE ACWR": RPE_AC_ratio, "Date": date_range})
map_df = pd.DataFrame({"map_polyline": all_detailed["map.polyline"]})

C:\Users\jackl\AppData\Local\Temp\ipykernel_13456\845607204.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RPE_workload['RPE'] = RPE_workload['RPE'].fillna(0)


In [15]:
i_all_detailed = all_detailed.interactive()
i_workload_df = workload_df.interactive()
workload_yaxis = pn.widgets.RadioButtonGroup(name='Yaxis', options=['Distance ACWR', 'RPE ACWR'], button_type='default')

In [17]:
workload_plot = i_workload_df.hvplot(x='Date',  y=workload_yaxis)
workload_plot